## Scikeras
---
- 설치 필요 : pip install scikeras

In [3]:
# !pip install scikeras

# 모듈 로딩

In [43]:
from scikeras.wrappers import KerasClassifier, KerasRegressor
from tensorflow.keras import Sequential
from tensorflow import keras
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import set_random_seed
from tensorflow.keras.backend import clear_session
from sklearn.cluster import DBSCAN
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [20]:
## 동일 결과 출력 시드 설정

In [21]:
clear_session()
np.random.seed(42)
set_random_seed(42)

## 데이터 로딩 및 준비

In [27]:
iris = load_iris()

In [28]:
X = iris['data']
y = iris['target']

In [29]:
# # 훈련/테스트 데이터 나누기
# x_train, x_test, y_train, y_test = train_test_split(X, y,
#                                                     random_state=42,
#                                                     stratify=y)

In [37]:
def get_model(hidden_layer_dim, meta):
#     # note that meta is a special argument that will be
#     # handed a dict containing input metadata
    n_features_in_ = meta["n_features_in_"]
    X_shape_ = meta["X_shape_"]
    n_classes_ = meta["n_classes_"]
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(n_features_in_, input_shape=X_shape_[1:]))
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.Dense(hidden_layer_dim))
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.Dense(n_classes_))
    model.add(keras.layers.Activation("softmax"))
    return model

clf = KerasClassifier(
    get_model,
    loss="sparse_categorical_crossentropy",
    hidden_layer_dim=100,
)

clf.fit(X, y)
y_proba = clf.predict_proba(X)

5/5 [==============================] - 0s 1ms/step


In [42]:
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('clf', clf),
])

pipe.fit(X, y)
y_proba = pipe.predict_proba(X)

5/5 [==============================] - 0s 740us/step


In [44]:
from sklearn.model_selection import GridSearchCV


params = {
    "hidden_layer_dim": [50, 100, 200],
    "loss": ["sparse_categorical_crossentropy"],
    "optimizer": ["adam", "sgd"],
    "optimizer__learning_rate": [0.0001, 0.001, 0.1],
}
gs = GridSearchCV(clf, params, refit=False, cv=3, scoring='accuracy')

gs.fit(X, y)
print(gs.best_score_, gs.best_params_)

2/2 [==============================] - 0s 997us/step
0.54 {'hidden_layer_dim': 100, 'loss': 'sparse_categorical_crossentropy', 'optimizer': 'adam', 'optimizer__learning_rate': 0.1}
